## Electricity Load Forecasting with Temperature

**Project goal:** Predicting the electricity load for a given hour ***t*** by leveraging how electricity consumption behaved at hourly intervals across years. Additionally, studying whether or not considering temperature data improves prediction.

**Project scope:** Information will be limited to a three-year time interval (2023-2025) and to Romania as location.

### Data Acquisition
**Electricity load** and **meteorological data** are not readily available in a single unified source, hence drawing from multiple sources is needed.
The former can be obtained from the **ENTSO-E Transparency Platform**, the official data portal of the European Network of Transmission System Operators for Electricity.

The default format provided by the ENTSO-E is quarter-hourly.

**Source:** https://transparency.entsoe.eu/

In [1]:
import pandas as pd

df_2023 = pd.read_csv("RO_2023.csv")
df_2024 = pd.read_csv("RO_2024.csv")
df_2025 = pd.read_csv("RO_2025.csv")
df = pd.concat([df_2023,df_2024,df_2025], ignore_index = True)
print("Rows and columns:", df.shape)
df.head()

Rows and columns: (105216, 4)


,MTU (CET/CEST),Area,Actual Total Load (MW),Day-ahead Total Load Forecast (MW)
0,01/01/2023 00:00 - 01/01/2023 00:15,Romania (RO),5074.0,5310.0
1,01/01/2023 00:15 - 01/01/2023 00:30,Romania (RO),5016.0,5220.0
2,01/01/2023 00:30 - 01/01/2023 00:45,Romania (RO),4963.0,5150.0
3,01/01/2023 00:45 - 01/01/2023 01:00,Romania (RO),4910.0,5090.0
4,01/01/2023 01:00 - 01/01/2023 01:15,Romania (RO),4881.0,5030.0


As electricity load information is provided by country, a country-wide estimate for hourly temperature based on a geographically representative selection of cities will suffice for the scope of this project.

**Meteorological** data for these cities will be manually retrieved from **Open-Meteo's** Historical Weather API page, using each city's coordinates.

Cities and coordinates:

Bucharest 44.43225, 26.10626 \
Cluj-Napoca 46.76667, 23.6 \
Iaşi 47.16667, 27.6 \
Constanţa 44.18073, 28.63432 \
Timişoara 45.75372, 21.22571 \
Braşov 45.64861, 25.60613 \
Craiova 44.31667, 23.8

**Source of coordinates:** https://www.geodatos.net/ \
**Source of data:** https://open-meteo.com/en/docs/historical-weather-api

In [2]:
df_temp_example = pd.read_csv("open-meteo-bucuresti.csv", skiprows = 2)
print("Rows and columns:", df_temp_example.shape)
df_temp_example.head()                             

Rows and columns: (26304, 2)


,time,temperature_2m (°C)
0,2023-01-01T00:00,4.3
1,2023-01-01T01:00,4.0
2,2023-01-01T02:00,4.4
3,2023-01-01T03:00,4.0
4,2023-01-01T04:00,4.1


### Data Preprocessing for the Electricity Load Dataset

In [3]:
# Extracting the calendaristic date and start hour for each electricity load

df["start_time_str"] = df["MTU (CET/CEST)"].str.split(" - ").str[0]
df[["MTU (CET/CEST)","start_time_str"]].head()

,MTU (CET/CEST),start_time_str
0,01/01/2023 00:00 - 01/01/2023 00:15,01/01/2023 00:00
1,01/01/2023 00:15 - 01/01/2023 00:30,01/01/2023 00:15
2,01/01/2023 00:30 - 01/01/2023 00:45,01/01/2023 00:30
3,01/01/2023 00:45 - 01/01/2023 01:00,01/01/2023 00:45
4,01/01/2023 01:00 - 01/01/2023 01:15,01/01/2023 01:00


In [4]:
# The format of the extracted start times is inconsistent: some dates contain 
# the string "CET" (Central European Time or "CEST" (Central European Summer Time)

df.loc[
    df["MTU (CET/CEST)"].str.contains("CET|CEST"),
    ["MTU (CET/CEST)", "start_time_str"]
].sample(3, random_state = 1)

,MTU (CET/CEST),start_time_str
98790,26/10/2025 02:30 (CEST) - 26/10/2025 02:45 (CEST),26/10/2025 02:30 (CEST)
98794,26/10/2025 02:30 (CET) - 26/10/2025 02:45 (CET),26/10/2025 02:30 (CET)
98788,26/10/2025 02:00 (CEST) - 26/10/2025 02:15 (CEST),26/10/2025 02:00 (CEST)


In [5]:
# Removing format inconsistencies caused by "CET"/"CEST"
df["start_time_str"] = df["start_time_str"].str.replace(r" \(CET\)| \(CEST\)","", regex = True)

# Converting the start time string to datetime and storing in column start_time
df["start_time"] = pd.to_datetime(df["start_time_str"], dayfirst = True)

df[["start_time_str","start_time"]].head()    

,start_time_str,start_time
0,01/01/2023 00:00,2023-01-01 00:00:00
1,01/01/2023 00:15,2023-01-01 00:15:00
2,01/01/2023 00:30,2023-01-01 00:30:00
3,01/01/2023 00:45,2023-01-01 00:45:00
4,01/01/2023 01:00,2023-01-01 01:00:00


In [6]:
df = df.set_index("start_time")
df.index[:5]

DatetimeIndex(['2023-01-01 00:00:00', '2023-01-01 00:15:00',
               '2023-01-01 00:30:00', '2023-01-01 00:45:00',
               '2023-01-01 01:00:00'],
              dtype='datetime64[ns]', name='start_time', freq=None)

The quarter-hourly values are aggregated into hourly loads in order to better inform the subsequent predictive model.

The ENTSO-E data provides power (measured in MegaWatts), which represents the rate at which electricity is being consumed, hence the appropriate way to aggregate quarter-hourly values to hourly is by computing the mean average for said hour.

In [7]:
hourly = df[["Actual Total Load (MW)","Day-ahead Total Load Forecast (MW)"]].resample("h").mean()
hourly.head()

,Actual Total Load (MW),Day-ahead Total Load Forecast (MW)
start_time,,
2023-01-01 00:00:00,4990.75,5192.5
2023-01-01 01:00:00,4814.50,4967.5
2023-01-01 02:00:00,4663.75,4787.5
2023-01-01 03:00:00,4566.75,4630.0
2023-01-01 04:00:00,4520.75,4580.0


In [8]:
hourly.index.min(), hourly.index.max(), hourly.shape

(Timestamp('2023-01-01 00:00:00'),
 Timestamp('2025-12-31 23:00:00'),
 (26304, 2))

### Data Preprocessing for the Temperature Dataset

In [9]:
# Creating a dictionary of "city name" -> corresponding dataframe

temp_files = {
    "Bucuresti": "open-meteo-bucuresti.csv",
    "Cluj-Napoca": "open-meteo-cluj.csv",
    "Iasi": "open-meteo-iasi.csv",
    "Constanta": "open-meteo-constanta.csv",
    "Timisoara": "open-meteo-timisoara.csv",
    "Brasov": "open-meteo-brasov.csv",
    "Craiova": "open-meteo-craiova.csv",
}

temp_dfs = {}

for city, fname in temp_files.items():
    df_temp = pd.read_csv(fname, skiprows = 2)
    df_temp["time"] = pd.to_datetime(df_temp["time"])
    df_temp = df_temp.rename(columns={"temperature_2m (°C)": "temp_c"})[["time", "temp_c"]]
    temp_dfs[city] = df_temp

{city: df_temp.shape[0] for city, df_temp in temp_dfs.items()}

{'Bucuresti': 26304,
 'Cluj-Napoca': 26304,
 'Iasi': 26304,
 'Constanta': 26304,
 'Timisoara': 26304,
 'Brasov': 26304,
 'Craiova': 26304}

In [10]:
temp_dfs["Bucuresti"].head()

,time,temp_c
0,2023-01-01 00:00:00,4.3
1,2023-01-01 01:00:00,4.0
2,2023-01-01 02:00:00,4.4
3,2023-01-01 03:00:00,4.0
4,2023-01-01 04:00:00,4.1


In [11]:
base_time = temp_dfs["Bucuresti"]["time"]
timestamps_match = all(df_temp["time"].equals(base_time) for df_temp in temp_dfs.values())
print("All timestamps are identical across cities:", timestamps_match)

All timestamps are identical across cities: True


In [12]:
# Building country-wide hourly average temperature table
temp_matrix = pd.DataFrame({city: df_temp["temp_c"].to_numpy() for city, df_temp in temp_dfs.items()})
avg_temp_c = temp_matrix.mean(axis=1)

temp_country = pd.DataFrame({
    "time":base_time,
    "temp_c_avg": avg_temp_c
})

temp_country.head()

,time,temp_c_avg
0,2023-01-01 00:00:00,5.657143
1,2023-01-01 01:00:00,5.000000
2,2023-01-01 02:00:00,5.171429
3,2023-01-01 03:00:00,4.500000
4,2023-01-01 04:00:00,4.857143


In [14]:
temp_country = temp_country.set_index("time")
temp_country.index.min(), temp_country.index.max(), temp_country.shape

(Timestamp('2023-01-01 00:00:00'),
 Timestamp('2025-12-31 23:00:00'),
 (26304, 1))

#### Merging Electrictiy Load and Temperature Datasets

In [16]:
merged = hourly.join(temp_country, how = "inner")
merged.head()

,Actual Total Load (MW),Day-ahead Total Load Forecast (MW),temp_c_avg
start_time,,,
2023-01-01 00:00:00,4990.75,5192.5,5.657143
2023-01-01 01:00:00,4814.50,4967.5,5.000000
2023-01-01 02:00:00,4663.75,4787.5,5.171429
2023-01-01 03:00:00,4566.75,4630.0,4.500000
2023-01-01 04:00:00,4520.75,4580.0,4.857143


In [17]:
merged.shape, merged.index.min(), merged.index.max()

((26304, 3),
 Timestamp('2023-01-01 00:00:00'),
 Timestamp('2025-12-31 23:00:00'))